In [3]:
import sys
from PIL import Image
sys.path.append("D:\Python_Projects\PhD_project")
# sys.path.insert(0, 'D:/Python_Projects/PhD_project')
import importlib
import os
import copy
import numpy as np
import matplotlib.pyplot as plt
import cv2
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, models, transforms
import torchvision
from visualDet3D.visualDet3D.data.kitti.utils import write_result_to_file
from visualDet3D.visualDet3D.utils.utils import LossLogger, cfg_from_file
from visualDet3D.visualDet3D.networks.utils.registry import DETECTOR_DICT, DATASET_DICT, PIPELINE_DICT
from visualDet3D.visualDet3D.networks.heads.anchors import Anchors
from visualDet3D.visualDet3D.networks.lib.fast_utils.hill_climbing import post_opt
from visualDet3D.visualDet3D.networks.utils import BBox3dProjector, BackProjection
from visualDet3D.visualDet3D.utils.utils import convertAlpha2Rot, convertRot2Alpha, draw_3D_box, compound_annotation
#from visualDet3D.visualDet3D.data.kitti.dataset import *
import visualDet3D.visualDet3D.data.kitti.dataset
from visualDet3D.visualDet3D.utils.timer import Timer
from numba import jit
from tqdm import tqdm
print('CUDA available: {}'.format(torch.cuda.is_available()))

cfg = cfg_from_file('D:/Python_Projects/PhD_project/visualDet3D/config/config.py')
is_test_train = True

checkpoint_name = 'D:/Python_Projects/PhD_project/visualDet3D/workdirs/Mono3D/checkpoint/GroundAware_pretrained.pth'

CUDA available: True


In [4]:
def draw_bbox2d_to_image(image, bboxes2d, color=(255, 0, 255)):
    drawed_image = image.copy()
    for box2d in bboxes2d:
        cv2.rectangle(drawed_image, (int(box2d[0]), int(box2d[1])), (int(box2d[2]), int(box2d[3])), color, 3)
    return drawed_image

In [11]:
from visualDet3D.visualDet3D.networks.detectors.yolomono3d_detector import GroundAwareYolo3D
cfg.batch_size=1
split_to_test='validation'

is_test_train = split_to_test == 'training'
if split_to_test == 'training':
    dataset_name = cfg.data.train_dataset
elif split_to_test == 'test':
    dataset_name = cfg.data.test_dataset
else:
    dataset_name = cfg.data.val_dataset

dataset = DATASET_DICT[dataset_name](
        cfg, split_to_test
        )

if split_to_test=='training':
    dataset_val = DATASET_DICT[cfg.data.val_dataset](
            cfg, 'validation'
            )
    dataset.transform = dataset_val.transform
    dataset.collate_fn = dataset_val.collate_fn

#detector = DETECTOR_DICT[cfg.detector.name](cfg.detector)
detector = GroundAwareYolo3D(cfg.detector)
detector = detector.cuda()

weight_path = os.path.join(cfg.path.checkpoint_path, checkpoint_name)
state_dict = torch.load(weight_path, map_location='cuda:{}'.format(cfg.trainer.gpu))
new_dict = state_dict.copy()
for key in state_dict:
    if 'focalLoss' in key:
        new_dict.pop(key)
detector.load_state_dict(new_dict, strict=False)
detector.eval().cuda()

# testing pipeline
test_func = PIPELINE_DICT[cfg.trainer.test_func]

projector = BBox3dProjector().cuda()
backprojector = BackProjection().cuda()

ModuleNotFoundError: No module named 'visualDet3D.data.kitti'